<a href="https://colab.research.google.com/github/ryanky31-code/model-training-gemma270m/blob/main/site/en/gemma/colab_quickstart_gemma_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemma-3 270M LoRA / QLoRA Quickstart (Colab)

This notebook helps you set up a Colab GPU runtime, install required packages, provide your Hugging Face token, choose a model variant, and run a small dry-run of the project's training CLI in `--mode lora` or `--mode qlora`.

Open the runtime type as `GPU` before running the install cell.

## 1) Install dependencies (run in a GPU runtime)
Run the cell below to install the core dependencies. For QLoRA you also need a compatible CUDA + bitsandbytes build.

In [ ]:
# Install core dependencies. This may take a few minutes in Colab.
!pip install --upgrade pip
!pip install transformers accelerate datasets
!pip install -r requirements-dev.txt
# bitsandbytes and peft are optional but useful for LoRA/QLoRA runs; the pip may fail on some envs, so allow failure
!pip install bitsandbytes peft || true

## 2) Provide your Hugging Face token
Run the cell and paste your HF token when prompted. The token is stored in the environment for the notebook session only.

In [ ]:
from getpass import getpass
import os
token = getpass('Enter your Hugging Face token (input hidden): ')
if token:
  os.environ['HF_TOKEN'] = token
  print('HF token set in environment for this session')
else:
  print('No token provided; some actions may be skipped')

## 3) Choose model variant and mode
Use the next cell to pick either `google/gemma-3-270m` (default) or another compatible Gemma checkpoint. Toggle QLoRA if you want to attempt 4-bit quantized flow (only if bitsandbytes is installed).

In [ ]:
# Default model id (change if you have a different HF checkpoint)
model_id = 'google/gemma-3-270m'
# Set to True to attempt QLoRA (requires bitsandbytes + CUDA)
use_qlora = False
# Example overrides: model_id = 'google/gemma-3-270m'
print(f'Model: {model_id}, QLoRA enabled: {use_qlora}')

## 4) Run smoke CSV generation and a small dry-run training check
This will generate the smoke CSV and run the CLI in `--dry-run` mode. No actual training will be performed in dry-run mode.

If you enabled QLoRA above and bitsandbytes is installed, the script will attempt to use QLoRA; otherwise it will fall back to LoRA or full-fine-tune scaffolding.

Adjust `max_rows` to a small number (e.g., 20) for a quick check.

In [ ]:
# Generate smoke CSV and run a dry-run training check
!python scripts/generate_synthetic_smoke.py
!python scripts/finetune_gemma_from_csv.py --csv synthetic_wifi_5ghz_outdoor_smoke.csv --mode lora --dry-run --max-rows 20 --target-field expected_throughput_mbps

## 5) Next steps
- If the dry-run shows dataset preparation and LoRA scaffolding, you can run a real LoRA training by removing `--dry-run` and setting small `--num-epochs` and `--per-device-batch-size`.
- For QLoRA, make sure the bitsandbytes installation succeeded and your runtime has a compatible CUDA version.